In [3]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/gdrive')
%cd /content/drive/MyDrive/github/intent_model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/github/intent_model


In [4]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), 629 bytes | 0 bytes/s, done.
From https://github.com/kmanoj845/intent_model
   169a94d..570826e  master     -> origin/master
Updating 169a94d..570826e
Fast-forward
 entity_data/entities-en.yaml | 40 ++++++++++++++++++++++++++++------------
 1 file changed, 28 insertions(+), 12 deletions(-)


### imports

In [5]:
!pip install indic-num2words

In [6]:
import os
import pickle
import numpy as np
import pandas as pd
import torch
import utils
# import importlib
# importlib.reload(utils)

lang_list = ['hi', 'en']

### Fill Placeholders

In [28]:
# for lang in lang_list:
#   for i in range(10):
#     utils.fill_placeholders(lang)

## Training

### Get manually created data

In [39]:
def get_training_data(lang_list):
    df = pd.DataFrame()
    # lang = 'en'
    all_data = pd.DataFrame()
    for lang in lang_list:
        read_dir = os.path.join(os.getcwd(),'formatted_data', lang)
        files = os.listdir(read_dir)
        data_files = [file for file in files if file.endswith(".csv")]

        for file in data_files:
            file_df = pd.read_csv(f'{os.path.join(read_dir, file)}')
            df = pd.concat([df, file_df], ignore_index=True)

    return df

In [32]:
data = get_training_data(lang_list)
data = data.rename(columns={"sentence": "Text", "intent": "Label", 'language': 'lang'})
print(f'before de-dupe, shape is {data.shape}')
data = data.drop_duplicates()
data = data.reset_index(drop=True)
print(f'after de-dupe, shape is {data.shape}')

before de-dupe, shape is (136500, 3)
after de-dupe, shape is (66927, 3)


In [36]:
# data.tail()

### Training utilities

In [38]:
# !pip install transformers
# !pip install sentencepiece
# !pip install transformers[torch]
# # !pip install accelerate -U

In [44]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EarlyStoppingCallback,
    Trainer,
    TrainingArguments,
)

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)
torch.cuda.empty_cache()

MODEL = "ai4bharat/indic-bert"

tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_data(texts):
    tokenized_inputs = tokenizer(
        texts, padding="max_length", max_length=512, truncation=True
    )
    return tokenized_inputs


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="micro")
    precision = precision_score(y_true=labels, y_pred=pred, average="micro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="micro")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


def train(data):
    df = data
    df = df.dropna()

    labels = sorted(list(set(df["Label"])))
    labels_to_ids = {k: v for v, k in enumerate(sorted(labels))}

    X = list(df["Text"])
    y = list(df["Label"])
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
    X_train_tokenized = tokenizer(
        X_train, padding=True, truncation=True, max_length=512
    )
    X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL, num_labels=len(labels)
    )
    train_dataset = utils.Dataset(X_train_tokenized, y_train, labels_to_ids)
    val_dataset = utils.Dataset(X_val_tokenized, y_val, labels_to_ids)
    output_dir = os.path.join(os.getcwd(),'models')

    train_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="steps",
        eval_steps=1000,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        seed=0,
        save_steps=1000,
        load_best_model_at_end=True,
    )
    trainer = Trainer(
        model=model,
        args=train_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
    with open(os.path.join(output_dir, "labels-dict.pkl"), "wb") as f:
        pickle.dump(labels_to_ids, f)

    train_metrics = trainer.train()
    eval_metrics = trainer.evaluate()

    print(train_metrics)
    print(eval_metrics)

## Train the model

In [45]:
# train(data)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1000,0.088700,0.062155,0.987375,0.987375,0.987375,0.987375
2000,0.021100,0.020900,0.995966,0.995966,0.995966,0.995966
3000,0.009900,0.010592,0.997684,0.997684,0.997684,0.997684
4000,0.003600,0.005189,0.998655,0.998655,0.998655,0.998655
5000,0.003000,0.005823,0.998805,0.998805,0.998805,0.998805


TrainOutput(global_step=5022, training_loss=0.11099627465365751, metrics={'train_runtime': 1601.0945, 'train_samples_per_second': 100.321, 'train_steps_per_second': 3.137, 'total_flos': 412877873337930.0, 'train_loss': 0.11099627465365751, 'epoch': 3.0})
{'eval_loss': 0.005189098417758942, 'eval_accuracy': 0.9986553115194979, 'eval_precision': 0.9986553115194979, 'eval_recall': 0.9986553115194979, 'eval_f1': 0.9986553115194979, 'eval_runtime': 41.1319, 'eval_samples_per_second': 325.441, 'eval_steps_per_second': 10.187, 'epoch': 3.0}


### Get GPT generated data

In [46]:
lang_list = ['hi', 'en']
def get_gpt_data(lang_list):
  df = pd.read_csv(os.path.join(os.getcwd(),'gpt_data', 'all-lang-all.csv'))
  df = df[df['lang'].isin(lang_list)]
  return df

In [47]:
gpt_data = get_gpt_data(lang_list)

In [76]:
# gpt_data.head()

In [69]:
import pickle

import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer
from utils import Dataset

MODEL = "ai4bharat/indic-bert"


def softmax(x):
    e_x = np.exp(x)
    return e_x / np.sum(e_x, axis=1, keepdims=True)


class IntentRecognizer:
    def __init__(self, model_path, label_dict_pkl, conf_threshold):
        with open(label_dict_pkl, "rb") as f:
            self.labels_to_ids = pickle.load(f)
        self.ids_to_labels = {
            intent_id: intent_label
            for intent_label, intent_id in self.labels_to_ids.items()
        }
        self.tokenizer = AutoTokenizer.from_pretrained(MODEL)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_path, num_labels=len(self.labels_to_ids)
        )
        self.test_trainer = Trainer(self.model)
        self.conf_threshold = conf_threshold

    def predict(self, sentence):
        sentence = [sentence]
        sentence_tokenized = self.tokenizer(
            sentence, padding=True, truncation=True, max_length=512
        )
        model_in = Dataset(sentence_tokenized)
        raw_pred, label_ids, _ = self.test_trainer.predict(model_in)
        # print(raw_pred, metrics)
        probs = softmax(raw_pred)
        # print(probs)
        y_pred = np.argmax(probs, axis=1)[0]
        pred_prob = np.max(probs)

        orig_pred = self.ids_to_labels[y_pred]
        # pred = self.ids_to_labels[y_pred]
        # if pred_prob < self.conf_threshold:
        #     pred = "unknown"
        return orig_pred, pred_prob

In [78]:
import pandas as pd
# from intent_recognizer import IntentRecognizer
from sklearn.metrics import classification_report
from tqdm import tqdm


def test(data):
    df = data
    df = df.dropna()
    model_path = os.path.join(os.getcwd(),'models', 'checkpoint-4000')

    intent_recognizer = IntentRecognizer(
        model_path, os.path.join(os.getcwd(),'models', 'labels-dict.pkl'), conf_threshold=0.85
    )

    true_intent = list()
    pred_intent = list()
    pred_probability = list()

    for i, row in tqdm(df.iterrows()):
        true_intent.append(row["Label"])
        intent, pred_prob = intent_recognizer.predict(row["Text"])
        pred_intent.append(intent)
        pred_probability.append(pred_prob)

    df["Predicted Intent"] = pred_intent
    df["Predicted Prob"] = pred_probability
    print(classification_report(true_intent, pred_intent))
    df.to_csv(os.path.join(os.getcwd(),'gpt_data', 'filtered_en_hi_data.csv'), index=False)
    return df


In [ ]:
filtered_data = test(gpt_data)
# print(filtered_data)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
0it [00:00, ?it/s]

6it [00:00, 50.64it/s]

12it [00:00, 50.66it/s]

18it [00:00, 51.35it/s]

24it [00:00, 51.51it/s]

...

17040it [07:05, 36.55it/s]

17044it [07:05, 35.13it/s]

17048it [07:05, 34.23it/s]

17052it [07:06, 34.46it/s]

17056it [07:06, 32.75it/s]